# Metrics of classification probabilities

TODO: change double float probs to integers, for less space/volume of data, could precompute from binned probabilities, check that it can be done in sql

also check the probabilities sum to 1 at some point
and allow submissions for negative of class
and flags for did not classify and reasons

In [ ]:
%matplotlib inline
import sys
import os
import requests
import datetime
import dateutil.parser
import json
import numpy
import pandas
import matplotlib.pyplot as plt
import sklearn
import sklearn.metrics

In [ ]:
 os.getenv("HOME")

In [ ]:
url = "https://desc-tom.lbl.gov"
# url = "https://desc-tom-rknop-dev.lbl.gov"

# Somehow you need to set the "username" and "password" variables.
# Do NOT hardcode the password into any code that will be world
# readable.  You can use the python input statement, or, as I do
# here, read the password from a file that is only readable by
# you.  (This isn't perfect, but it's better than having the
# password sit around in a world readable file.)
username = "aimalz"#"rknop"
with open( os.path.join( os.getenv("HOME"), "secrets", "tom_"+username+"_passwd" ) ) as ifp:
    password = ifp.readline().strip()

In [ ]:
rqs = requests.session()
rqs.get( f'{url}/accounts/login/' )
res = rqs.post( f'{url}/accounts/login/',
                data={ "username": username,
                       "password": password,
                       "csrfmiddlewaretoken": rqs.cookies['csrftoken'] } )
if res.status_code != 200:
    raise RuntimeError( f"Failed to log in; http status: {res.status_code}" )
if 'Please enter a correct' in res.text:
    raise RuntimeError( "Failed to log in.  I think.  Put in a debug break and look at res.text" )
rqs.headers.update( { 'X-CSRFToken': rqs.cookies['csrftoken'] } )

In [ ]:
def run_query(query, subdict=None):
    if subdict == None:
        subdict = {}
    result = rqs.post( f'{url}/db/runsqlquery/',
        json={ 'query': query, 'subdict': subdict } )
    if result.status_code != 200:
        sys.stderr.write( f"ERROR: got status code {result.status_code} ({result.reason})\n" )
    else:
        data = json.loads( result.text )
        if ( 'status' not in data ) or ( data['status'] != 'ok' ):
            sys.stderr.write( "Got unexpected response\n" )
            print(data['error'])
        else:
            return data['rows']

In [ ]:
rows = run_query( 'SELECT "classifierId","brokerName","brokerVersion",'
                  '"classifierName","classifierParams" '
                  'FROM elasticc_brokerclassifier' )
classifier_info = {}
for row in rows:
    classifier_info[row["classifierId"]] = row             
classifier_info

TODO: select just a couple classifiers/brokers

In [ ]:
use_cfers = {i: classifier_info[i] for i in [40,44]}
use_cfers

In [ ]:
rows = run_query( 'SELECT DISTINCT ON ("classId") "classId",description '
                  'FROM elasticc_gentypeofclassid '
                  'ORDER BY "classId"' )
classname = {}
for row in rows:
    classname[row["classId"]] = row["description"]

In [ ]:
# consider descIngestTimestamp, alertSentTimestamp
#
# alertSentTimestamp -- this gets updated in the database when the original
#    alert is sent out.  It's NOT present in the dev database, but should
#    be in the real one
# descIngestTimestamp -- just when we happend to suck in the broker message
#
# alertId -- opaque fingerprint, don't sort on it, but it does correspond to
#   the original alert.  Different alertId = different source, i.e.
#   measurement of the transient.

sys.stderr.write( f"Query start at {datetime.datetime.now().isoformat()}...\n" )
query = ( 'SELECT '
          '  "classifierId","diaObjectId","classId","alertId","trueClassId","probability","descIngestTimestamp" '
          'FROM elasticc_view_sourceclassifications '
          'ORDER BY "diaObjectId","descIngestTimestamp","classifierId" '
          'OFFSET 100 LIMIT 100' )
rows = run_query( query )
sys.stderr.write( f"Query done at {datetime.datetime.now().isoformat()}\n" )
data = pandas.DataFrame( rows )
data
# classifiers = data.classifierId.unique()

TODO: change double float probs to integers, for less space/volume of data
also check the probabilities sum to 1 at some point
and allow submissions for negative of class
and flags for did not classify and reasons

In [ ]:
classname

In [ ]:
sys.stderr.write( f"Query start at {datetime.datetime.now().isoformat()}...\n" )
query = ( 'SELECT '
          '  "classifierId","diaObjectId","classId","alertId","trueClassId","probability","descIngestTimestamp","alertSentTimestamp" '
          'FROM elasticc_view_sourceclassifications '
          'WHERE "classifierId"=41 AND "classId"=111 AND "trueClassId"=111 '
          'ORDER BY "diaObjectId","descIngestTimestamp" '
          'OFFSET 100 LIMIT 100' )
rows = run_query( query )
sys.stderr.write( f"Query done at {datetime.datetime.now().isoformat()}\n" )
data = pandas.DataFrame( rows )
data

In [ ]:
import numpy as np
np.unique(data['diaObjectId'], return_counts=True)

In [ ]:
chosen = 1006052#1003722

In [ ]:
data[ data['diaObjectId'] == chosen]#1025381 ]

In [ ]:
# All these joins suggest we need a new view.
sys.stderr.write( f"Query start at {datetime.datetime.now().isoformat()}...\n" )
query = ( 'SELECT '
          '  v."classifierId",v."diaObjectId",v."classId",v."alertId",v."trueClassId",v."probability",v."alertSentTimestamp", '
          '  s."midPointTai" '
          'FROM elasticc_view_sourceclassifications v '
          'INNER JOIN elasticc_diasource s ON v."diaSourceId"=s."diaSourceId" '
          'WHERE v."diaObjectId"='+str(chosen)+' AND "classId"=111 AND "trueClassId"=111 '
          'ORDER BY "classifierId","alertSentTimestamp" ' )
rows = run_query( query )
sys.stderr.write( f"Query done at {datetime.datetime.now().isoformat()}\n" )
data = pandas.DataFrame( rows )
data

In [ ]:
pandas.set_option('display.max_rows', 500)
data['t'] = data['alertSentTimestamp'].apply( lambda thing : dateutil.parser.isoparse( thing ) )
data.columns


In [ ]:
classifier_info

In [ ]:
for i,cfer in enumerate(classifiers):
    print(cfer,classifier_info[cfer]["brokerName"],classifier_info[cfer]["classifierName"],classifier_info[cfer]["brokerVersion"])

In [ ]:
blind = False

In [ ]:
    
# TODO: color for broker, linestyle for classifier

fig = plt.figure( figsize=(12,8), tight_layout=True )
ax = fig.add_subplot( 1,1,1)
classifiers = data['classifierId'].unique()
for i,cfer in enumerate(use_cfers):
    subdata = data[ data['classifierId'] == cfer ]
    label = f'{use_cfers[cfer]["brokerName"]} {use_cfers[cfer]["classifierName"]}'
    ax.plot( subdata['midPointTai'], subdata['probability'], color=f'C{i}', label=label )
    ax.tick_params( "both", labelsize=16 )
if ~blind:
    ax.legend()
    ax.set_title('diaObjectID='+str(chosen))#"Jackson Pollock" )
ax.set_xlabel('midPointTai (time)', fontsize=16)
ax.set_ylabel('probability of true class', fontsize=16)
fig.show()

1. metric as f(time relative to characteristic time)
2. metric as f(number of alerts)



In [ ]:
# Oh, the humanity
sys.stderr.write( f"Query start at {datetime.datetime.now().isoformat()}...\n" )
query = ( 'SELECT '
          '  v."classifierId",v."diaObjectId",v."classId",v."alertId",v."probability",v."alertSentTimestamp", '
          '  s."midPointTai",t.peakmjd '
          'FROM elasticc_view_sourceclassifications v '
          'INNER JOIN elasticc_diasource s ON v."diaSourceId"=s."diaSourceId" '
          'INNER JOIN elasticc_diaobjecttruth t ON v."diaObjectId"=t."diaObjectId" '
          'WHERE v."diaObjectId" IN '
          '  ( SELECT "diaObjectId" FROM elasticc_diaobjecttruth WHERE gentype IN '
          '     ( SELECT gentype FROM elasticc_gentypeofclassid WHERE "classId"=111 ) '
          '    LIMIT 100 ) '
          'AND "classId"=111 '
          'ORDER BY "classifierId","alertSentTimestamp" ' )
rows = run_query( query )
sys.stderr.write( f"Query done at {datetime.datetime.now().isoformat()}\n" )
data = pandas.DataFrame( rows )
data

In [ ]:
data['Δt'] = data['midPointTai'] - data['peakmjd']
data

In [ ]:
onebroker = data[ data['classifierId'] == 40 ].copy()

In [ ]:
onebroker

In [ ]:
plt.hist( onebroker['Δt'] )

In [ ]:
# This is not the right way to bin.  We have to think harder about this.
# If there are more observations of some objectds, those objects will
# get overweighted.
onebroker['tbin'] = numpy.floor( ( onebroker['Δt'] + 20 ) / 5. )

In [ ]:
thing = onebroker.groupby( 'tbin' )['probability'].apply( 'mean' )

In [ ]:
plt.plot(thing)